In [1]:
# Environ
import scipy as scp
import tensorflow as tf
from scipy.stats import gamma
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import random
import multiprocessing as mp
import psutil
import pickle
import os
import re

# Own
import ddm_data_simulation as ds
import cddm_data_simulation as cds
import kde_training_utilities as kde_util
import kde_class as kde
import boundary_functions as bf

In [26]:
# DDM  
repeats = 1
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.ddm_flexbound(v = 0, 
                            a = 0.96,
                            w = 0.5,
                            ndt = 0.5,
                            delta_t = 0.01, 
                            max_t = 20,
                            n_samples = 20000,
                            boundary_fun = bf.angle,
                            boundary_multiplicative = False, 
                            boundary_params = {"theta": 0.01})
    print(i)
    
    my_means[i] = np.mean(out[0][out[1] == 1])

0


In [39]:
def bin_simulator_output(out = [0, 0],
                         bin_dt = 0.04,
                         eps_correction = 1e-7):

    # hardcode 'max_t' to 20sec for now
    n_bins = int(20.0 / bin_dt + 1)
    #n_bins = int(out[2]['max_t'] / bin_dt + 1)
    bins = np.linspace(0, out[2]['max_t'], 257)
    counts = []
    counts.append(np.histogram(out[0][out[1] == 1], bins = bins)[0] / out[2]['n_samples'])
    counts.append(np.histogram(out[0][out[1] == -1], bins = bins)[0] / out[2]['n_samples'])

    n_small = 0
    n_big = 0

    for i in range(len(counts)):
        n_small += sum(counts[i] < eps_correction)
        n_big += sum(counts[i] >= eps_correction)

    for i in range(len(counts)):
        counts[i][counts[i] <= eps_correction] = eps_correction
        counts[i][counts[i] > eps_correction] = counts[i][counts[i] > eps_correction] - (eps_correction * (n_small / n_big))    

    for i in range(len(counts)):
        counts[i] =  np.asmatrix(counts[i]).T

    label = np.concatenate(counts, axis = 1)
    features = [out[2]['v'], out[2]['a'], out[2]['w']] # out[2]['ndt']]
    return (features, label)

In [40]:
#%%timeit -n 1 -r 5
a, b = bin_simulator_output(out = out)

In [41]:
b.shape

(256, 2)

In [5]:
files_ = os.listdir('/media/data_cifs/afengler/data/kde/ddm/base_simulations_20000')

In [8]:
labels = np.zeros((250000, 500, 2))
features = np.zeros((250000, 3))

cnt = 0
i = 0
file_dim = 100
for file_ in files_[:1000]:
    if file_[:8] == 'ddm_flex':
        out = pickle.load(open('/media/data_cifs/afengler/data/kde/ddm/base_simulations_20000/' + file_, 'rb'))
        features[cnt], labels[cnt] = bin_simulator_output(out = out)
        if cnt % file_dim == 0:
            print(cnt)
            pickle.dump((labels[(i * file_dim):((i + 1) * file_dim)], features[(i * file_dim):((i + 1) * file_dim)]), open('/media/data_cifs/afengler/data/kde/ddm/base_simulations_20000_binned/dataset_' + str(i), 'wb'))
            i += 1
        cnt += 1


0
100
200
300
400
500
600
700
800
900


In [19]:
files_[0:10]

['ddm_flexbound_8f20ccdcb4a211e9bfa0b92c7f3bbbb3.pickle',
 'ddm_flexbound_6e62f94cb4a811e9a2296161dd1187d4.pickle',
 'ddm_flexbound_471fbd92b4a411e985f6ffa9521cc62b.pickle',
 'ddm_flexbound_121436fcb49311e99f50856f5dfd3c68.pickle',
 'ddm_flexbound_94a3b878b49211e9993d4b8b0deebcde.pickle',
 'ddm_flexbound_82348a9eb49411e9a54c156d8f74be92.pickle',
 'ddm_flexbound_4c553792b49f11e9b0187b49e7ec8e30.pickle',
 'ddm_flexbound_150fda5cb4af11e9b63dc36fa94c651c.pickle',
 'ddm_flexbound_1185fc1eb4a411e9a92c59bea83f0590.pickle',
 'ddm_flexbound_3aa90a00b49f11e9b54705d8231d6519.pickle']

In [20]:
random.shuffle(files_)
files_[0:10]

['ddm_flexbound_5837b522b4a811e9811e5bf9d656495d.pickle',
 'ddm_flexbound_e4cfa5b0b49111e9b54705d8231d6519.pickle',
 'ddm_flexbound_7299cb8cb49c11e9ab6a19eb39c3d16e.pickle',
 'ddm_flexbound_6bfef1d6b4aa11e997417d415d78197e.pickle',
 'ddm_flexbound_fb9c7be0b4a711e991da87d516367dec.pickle',
 'ddm_flexbound_c9959f84b49111e9acc87dc8e438791b.pickle',
 'ddm_flexbound_35ea3ffeb4a211e99fc52daf56a52c95.pickle',
 'ddm_flexbound_d5c471eab49111e988acb30e98ec4b9a.pickle',
 'ddm_flexbound_7da11050b4a911e9ab0afba0ff2c72f3.pickle',
 'ddm_flexbound_bafec368b4a211e9b8291bebff384457.pickle']

In [ ]:
# FULL DDM  
repeats = 50
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.full_ddm(v = 0, 
                       a = 0.96,
                       w = 0.5,
                       ndt = 0.5,
                       dw = 0.0,
                       sdv = 0.0,
                       dndt = 0.5,
                       delta_t = 0.01, 
                       max_t = 20,
                       n_samples = 10000,
                       boundary_fun = bf.constant,
                       boundary_multiplicative = True, 
                       boundary_params = {})
    print(i)
    
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
plt.hist(my_means, density = True)

In [ ]:
np.mean(out[0][out[1] == 1])

In [ ]:
plt.hist(out[0] * out[1], bins = 50)

In [ ]:
int(50 / out[2]['delta_t'] + 1)

In [ ]:
# LCA 
repeats = 1
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.lca(v = np.array([0, 0], dtype = np.float32), 
                  a = 2, 
                  w = np.array([0.5, 0.5], dtype = np.float32), 
                  ndt = np.array([1.0, 1.0], dtype = np.float32),
                  g = -1.0,
                  b = 1.0,
                  delta_t = 0.01, 
                  max_t = 40,
                  n_samples = 10000,
                  boundary_fun = bf.constant,
                  boundary_multiplicative = True, 
                  boundary_params = {})
    print(i)
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
out[1][out[1] == 0] = -1
plt.hist(out[0] * out[1], bins = 50)

In [ ]:
# LCA 
repeats = 10
my_means = np.zeros(repeats)
for i in range(repeats):
    out = cds.ddm_flexbound(v = 0.0, 
                            a = 1.5, 
                            w = 0.5, 
                            ndt = 0.1,
                            delta_t = 0.01, 
                            max_t = 40,
                            n_samples = 10000,
                            boundary_fun = bf.constant,
                            boundary_multiplicative = True, 
                            boundary_params = {})
    print(i)
    my_means[i] = np.mean(out[0][out[1] == 1])

In [ ]:
plt.hist(my_means, density = True)

In [ ]:
out[1][out[1] == -1].shape

In [ ]:
int((40 / 0.01) + 1)

In [ ]:
my_bound = bf.constant

In [ ]:
my_bound

In [ ]:
my_bound(t = 100)

In [ ]:
h_Test = (1, 1, 1)

In [ ]:
h_Test + (1,2)

In [ ]:
h = ({'c': 1}, {'c2': 2})

In [ ]:
h_Test + h

In [ ]:
a = ({'c': 1},)

In [ ]:
h_Test + a

In [ ]:
def foo(name, *args, **kwargs):
    print ("args: ", args)
    print ("Type of args: ", type(args))
    if len(args)>2:
        args = args[0], args[1]     #- Created Same name variable.
    print ("Temp args:", args)

In [ ]:
foo('hello', ('a', 'b', 'c'))

In [ ]:
t = ('a', 'b', 'c')

In [ ]:
t[0]

In [ ]:
t2 = t[0], t[1]

In [ ]:
t2

In [ ]:
a = ['a']
a = a + ['b', 'c']

In [ ]:
a

In [ ]:
test_dat = pd.DataFrame(np.zeros((10, 5)), columns = ['v_0', 'v_1', 'a', 'b', 'c'])

In [ ]:
test_dat

In [ ]:
np.array(test_dat.loc[1, ['v_0', 'v_1']])

In [ ]:
my_keys = []
for key in test_dat.keys():
    if key[0] == 'v':
        my_keys.append(key)
np.array(test_dat.loc[1, ['v_0', 'v_1']])

In [ ]:
my_target_folder = '/media/data_cifs/afengler/tmp/'

In [ ]:
import make_data_lba as mdlba

In [ ]:
my_dat = mdlba.make_data_rt_choice(target_folder = my_target_folder)

In [ ]:
a = [1,2,3,4,5]

In [ ]:
a[:-2]

In [ ]:
np.isclose(0, 1e-15)

In [ ]:
1.0 - 1e-29 == 1.0

In [ ]:
np.max(my_dat['log_likelihood'])

In [ ]:
data = np.concatenate([out[0], out[1]], axis = 1)

In [ ]:
data[data[:,0] < 3, 0]

In [ ]:
np.log(1e-100)

In [ ]:
t = np.random.uniform(size = (1000, 10))

In [ ]:
t[:,2] = t[:, 1] + np.random.uniform(size = 1000) / t[:, 3]

In [ ]:
(np.random.uniform(size = (1000)) / t[:, 3]).shape

In [ ]:
t[:, 2]